# import

In [1]:
import numpy as np
import glob
import pandas as pd
import matplotlib.pyplot as plt
from my_library.library import *
from pprint import pprint
import json
import requests
import sys
path_ =  '/Users/Owner/Desktop/bitflyer_api'
sys.path.append(path_)
from settings import API_settings as bapi


In [13]:
path_win1 = '/Users/Owner/Desktop/BitCoin/Data/btc_day_chart.csv'
path_win2= '/Users/Owner/Desktop/BitCoin/Data/btc_min_chart.csv'

In [3]:
def load_csv(load_path):
    df = pd.read_csv(load_path, header=None)
    df = df.rename(columns={
    df.columns[0]:'day',
    df.columns[1]:'open',
    df.columns[2]:'high',
    df.columns[3]:'low',
    df.columns[4]:'close',
    df.columns[5]:'volume'
    })
    df['day'] = pd.to_datetime(df['day'],format='%Y/%m/%d')
    df.set_index('day',inplace=True)
    return df

In [14]:
df = load_csv(path_win2)

In [ ]:
ts = TechnicalSimulation(hold_day=100)
ts.simulate(df,start_year=2021,end_year=2022)

# bitflyer api

In [2]:
import ccxt
import datetime

In [3]:
bitflyer = ccxt.bitflyer({
'apiKey': bapi.API_KEY,
'secret': bapi.API_SECRET,
})

COIN = 'BTC'
PAIR = 'BTCJPY28SEP2018'

# ロット(単位はBTC)
LOT = 0.001

# 最小注文数(取引所の仕様に応じて設定)
AMOUNT_MIN = 0.001

# スプレッド閾値
SPREAD_ENTRY = 0.0005  # 実効スプレッド(100%=1,1%=0.01)がこの値を上回ったらエントリー
SPREAD_CANCEL = 0.0003 # 実効スプレッド(100%=1,1%=0.01)がこの値を下回ったら指値更新を停止

# 数量X(この数量よりも下に指値をおく)
AMOUNT_THRU = 0.01

# 実効Ask/BidからDELTA離れた位置に指値をおく
DELTA = 1

In [4]:

#------------------------------------------------------------------------------#
#log設定
import logging
logger = logging.getLogger('LoggingTest')
logger.setLevel(10)
fh = logging.FileHandler('log_mm_bf_' + datetime.datetime.now().strftime('%Y%m%d') + '_' + datetime.datetime.now().strftime('%H%M%S') + '.log')
logger.addHandler(fh)
sh = logging.StreamHandler()
logger.addHandler(sh)
formatter = logging.Formatter('%(asctime)s: %(message)s', datefmt="%Y-%m-%d %H:%M:%S")
fh.setFormatter(formatter)
sh.setFormatter(formatter)

#------------------------------------------------------------------------------#

# JPY残高を参照する関数
def get_asset():

    while True:
        try:
            value = bitflyer.fetch_balance()
            break
        except Exception as e:
            logger.info(e)
            time.sleep(1)
    return value



In [6]:
# JPY証拠金を参照する関数
def get_colla():

    while True:
        try:
            value = bitflyer.privateGetGetcollateral()
            break
        except Exception as e:
            logger.info(e)
            time.sleep(1)
    return value

In [7]:
from pprint import pprint
import json
import requests
BASE_URL = 'https://coincheck.com'
url = BASE_URL + '/api/order_books'
params = {
    'limit': 1
}
r = requests.get(url, params=params)
r = r.json()
pprint(r)

{'asks': [['2825400.0', '1.913']], 'bids': [['2825084.0', '0.0885']]}


In [16]:
ask = float(r['asks'][0][0])
bid = float(r['bids'][0][0])

In [17]:
spread = (ask - bid )/bid

In [18]:
spread

0.00011185508112325155